In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

#Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64 #for images


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

#variables to connect to database
USER = "aacuser"
PASS = "CS340"
HOST = "nv-desktop-services.apporto.com"
PORT = 31350
DB = "aac"
COL = "animals"
shelter = AnimalShelter(USER, PASS, HOST, PORT, DB, COL)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Adela Martinez | SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        # Code for the interactive filtering options (radio buttons)
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'RESET'},
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN OR WILDERNESS'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER OR INDIVIDUAL TRACKING'}
            ]
        )

    ),
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height':'10%','width':'10%'}
        ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = True,
        row_selectable = "single", #ability to select a single row
        selected_rows=[],
        sort_action = "native", #ability to sort data
        page_action="native", #enable pages
        page_current = 0,
        page_size = 10, # rows per page
        filter_action = "native" #ability to filter data
),
     html.Br(),
     html.Hr(),
     html.Div(className='row',
          style={'display' : 'flex', 'justify-content':'center'},
             children=[
         html.Div(
            id='graph-id',
            className='col s12 m6',

            ), 
         html.Div(
            id='map-id',
            className='col s12 m6',
            )
])
])

#############################################
# Interaction Between Components / Controller
#############################################
# Interaction Between Components / Controller
#############################################
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type): #queries for radio buttons
    if filter_type == 'RESET':
        filtered_df = df
    elif filter_type == 'WATER':
        filtered_df = df[df['animal_type'] == 'Dog']
        filtered_df = filtered_df[filtered_df['breed'].isin(["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"])]
        filtered_df = filtered_df[filtered_df['sex_upon_outcome'] == "Intact Female"]
        filtered_df = filtered_df[(filtered_df['age_upon_outcome_in_weeks'] >= 26) & (filtered_df['age_upon_outcome_in_weeks'] <= 156)]
    
    elif filter_type == 'MOUNTAIN OR WILDERNESS':
        filtered_df = df[df['animal_type'] == 'Dog']
        filtered_df = filtered_df[filtered_df['breed'].isin(["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"])]
        filtered_df = filtered_df[filtered_df['sex_upon_outcome'] == "Intact Male"]
        filtered_df = filtered_df[(filtered_df['age_upon_outcome_in_weeks'] >= 26) & (filtered_df['age_upon_outcome_in_weeks'] <= 156)]
        
        
    elif filter_type == 'DISASTER OR INDIVIDUAL TRACKING':
        filtered_df = df[df['animal_type'] == 'Dog']
        filtered_df = filtered_df[filtered_df['breed'].isin(["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                          "Bloodhound", "Rottweiler"])]
        filtered_df = filtered_df[filtered_df['sex_upon_outcome'] == "Intact Male"]
        filtered_df = filtered_df[(filtered_df['age_upon_outcome_in_weeks'] >= 20) & (filtered_df['age_upon_outcome_in_weeks'] <= 300)]
    else:
        filtered_df = df  # Default case, no filter applied
        
    data = filtered_df.to_dict('records')
    columns = [{'name': col, 'id': col} for col in filtered_df.columns]
    
    return data, columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if not viewData:
        return []  # Return an empty graph if there's no data

    dff = pd.DataFrame.from_dict(viewData)
    
    if 'breed' not in dff.columns:
        return []  # Return an empty graph if 'breed' column is not found

    return [
        dcc.Graph(            
            figure=px.pie(dff, names='breed', title='Preferred Animal Breeds')
        )
    ]
            


#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []  # Return an empty list if no columns are selected
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or not viewData:
        return []  # Return an empty map if there's no data

    if index is None or not index:
        return []  # Return an empty map if no row is selected

    dff = pd.DataFrame.from_dict(viewData)

    row = index[0] if index else 0  # Get the selected row index, default to 0 if no row is selected

    if row >= len(dff):
        return []  # Return an empty map if the selected row index is out of range

    return [
        dl.Map(style={'width': '400px', 'height': '200px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row, 4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row, 9])
                                 ])
                             ])
               ])
    ]



    

app.run_server(debug=True)

Dash app running on http://127.0.0.1:20008/
